In [1]:
type ∨[P, Q] = Either[P, Q] 
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzles Introducción

Primero se consideran 5 preguntas que servirán como introduccion a la logica 
knight-knave para aquellos que no estén familiarizados con ella y como un
breve curso recordatorio para los que lo están.

#### 3. Suppose an inhabitant A says about himself and his brother B: "At least one of us is a knave." What type is A and what type is B?

A says that, of A and B, at least one is a knave. If A were a knave, then it would be true that at least one of A and B is a knave and we would have a knave making a true statement, which is not possible. Therefore A must be a
knight. Since he is a knight, his statement is true, hence at least one really is a knave. It is then B who must be the knave.
So A is a knight and B is a knave.

In [5]:
def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = ???
    /*
    O X es Knave o Y es Knave. En la premisa solo dice que al menos uno es Knave. No dice al menos uno es Knave y el otro
    es Knight o viceversa.
    
    Al menos uno de ellos es un Knave (uno u otro):
    1.- X es Knave
    2.- Y es Knave
    
    "Uno u otro" -> Either[(x.Knave,y.Knave)]
    
    */



defined function puzzle3

In [6]:
//VERSION 2

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays : x.Says[Either[x.Knave, y.Knave]] => 
            ??? : (x.Knight,y.Knave)})

defined function puzzle3

In [7]:
//VERSION 2

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays : x.Says[Either[x.Knave, y.Knave]] => 
            (premises.P1(x) match{
                case Left(xKnight: x.Knight) =>
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    (xKnight: x.Knight, (x_3L match{
                        case Left(xKnave: x.Knave) => 
                            val aux_contradiccion: x.Knight => Nothing = xKnave
                            aux_contradiccion(xKnight) : y.Knave
                        case Right(yKnave: y.Knave) => 
                            yKnave : y.Knave
                    }): y.Knave) : (x.Knight,y.Knave)
                                
                case Right(xKnave: x.Knave) =>
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = premises.P3[Either[x.Knave, y.Knave]](x)
                    val x_2R: x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = x_1R(xKnave)
                    //val x_3R: Not[Either[x.Knave, y.Knave]] = x_2R(xSays)
                
                    val x_3R: Either[x.Knave, y.Knave] => Nothing = x_2R(xSays)
                    //Si a la funcion x_3R le pasas o un x.Knave o un y.Knave devuelve Nothing 
                    //Aqui tengo P que es "Either[x.Knave, y.Knave]" y tengo Not[P] que es "x_2R(xSays)",
                    //por tanto he demonstrado la contradiccion
                    x_3R(Left(xKnave)): Nothing
               
                (x_3R(Left(xKnave)), x_3R(Left(xKnave))) : (x.Knight,y.Knave)
                
            }) : (x.Knight,y.Knave)})

defined function puzzle3

In [8]:
//VERSION 2 - SIMPLIFICADA 

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays => 
            (premises.P1(x) match{
                case Left(xKnight) =>
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    x_3L match{
                        case Left(xKnave: x.Knave) => xKnave(xKnight) : (x.Knight,y.Knave)
                        case Right(yKnave: y.Knave) => (xKnight, yKnave) : (x.Knight,y.Knave)
                    }
                case Right(xKnave) =>
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = premises.P3[Either[x.Knave, y.Knave]](x)
                    val x_2R: x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = x_1R(xKnave)
                    val x_3R: Either[x.Knave, y.Knave] => Nothing = x_2R(xSays)
                                   
                    //(x_3R(Left(xKnave)), x_3R(Left(xKnave)))
                    x_3R(Left(xKnave)): (x.Knight,y.Knave)
            }) 
        })

defined function puzzle3

In [8]:
//Sugar version

/*def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays => 
            (premises.P1(x) match{
                case Left(xKnight) =>
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    x_3L match{
                        case Left(xKnave: x.Knave) => xKnave(xKnight) : (x.Knight,y.Knave)
                        case Right(yKnave: y.Knave) => (xKnight, yKnave) : (x.Knight,y.Knave)
                    }
                case Right(xKnave) =>
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = premises.P3[Either[x.Knave, y.Knave]](x)
                    val x_2R: x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = x_1R(xKnave)
                    val x_3R: Either[x.Knave, y.Knave] => Nothing = x_2R(xSays)
                                   
                    (x_3R(Left(xKnave)), x_3R(Left(xKnave)))
            }) 
        })
*/

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        xSay_xKnaveOryKnave =>
        //({xSays =>
            premises.eitherKnightOrKnave(x).fold(
            //(premises.P1(x) match{
                xIsKnight =>
                //case Left(xKnight) =>
                    val aux1 = premises.noKnightLies(x)(xIsKnight)(xSay_xKnaveOryKnave: x.Says[Either[x.Knave, y.Knave]]): Either[x.Knave, y.Knave]
                    aux1 match{
                        xKnave => xKnave(xIsKnight)
                        yKnave => (xIsKnight, yKnave)
                    }              
                ,
                xIsKnave =>
                //case Right(xKnave) =>
                    val aux2 = premises.no KnaveTellsTruth(x)(xIsKnave)(xSay_xKnaveOryKnave: x.Says[Either[x.Knave, y.Knave]]): Not[x.Says[Either[x.Knave, y.Knave]]]
                    aux2(Left(xIsKnave))            
            
            )

(console):35:21 expected ")"
//Sugar version

/*def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays => 
            (premises.P1(x) match{
                case Left(xKnight) =>
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    x_3L match{
                        case Left(xKnave: x.Knave) => xKnave(xKnight) : (x.Knight,y.Knave)
                        case Right(yKnave: y.Knave) => (xKnight, yKnave) : (x.Knight,y.Knave)
                    }
                case Right(xKnave) =>
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] =>

: 